VIII Evaluation
==

In [5]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%matplotlib inline
import numpy as np
from numpy import exp, sqrt, log, pi
from scipy.optimize import curve_fit
from scipy.stats import chi2
from scipy import odr
from scipy.integrate import quad
from scipy.signal import find_peaks
from scipy.signal import argrelextrema, argrelmin, argrelmax


def fehler(name, G, sig_G, G_lit, sig_G_lit):
    print(name)
    print('Relativer Fehler: ', sig_G / G * 100)
    print('Rel. Fehler (Vergleich):', sig_G_lit / G_lit * 100)
    print('Absoluter Fehler: ', G - G_lit)
#    print('Verhältnis:', G / G_lit)
    print('Sigma-Abweichung: ', np.abs(G - G_lit) / sqrt(sig_G ** 2 
                                                         + sig_G_lit ** 2),'\n')
    
    
def fehler_small(name, G, sig_G):
    print(name)
    print('Relativer Fehler: ', sig_G / G * 100)
    

def ergebnis(name, G, sig_G, komma, einheit):
    print(name + ' =', np.round(G, komma), '+/-', np.round(sig_G, komma), einheit)
    

def ergebnis_large(name, G, sig_G, komma, einheit):
    print(name + ' =', np.round(G, komma))
    print('+/-'.rjust(len(name) + 2), np.round(sig_G, komma), einheit)
    

def fitparameter(name, G, sig_G, komma, einheit):
    return name + ' =' + str(np.round(G, komma)) + '$\pm$' + str(np.round(sig_G, komma)) + einheit

Frequencies
---

In [6]:
# Experimental values
f_free = 0.62 # Hz
sig_f_free = 0.005

f_1 = np.array([0.620, 0.621, 0.621]) 
sig_f_1 = np.ones(3) * 0.005

f_2 = np.array([0.655, 0.680, 0.737])
sig_f_2 = np.array([0.010, 0.005, 0.011])

f_s1 = np.array([0.622, 0.621, 0.620])
sig_f_s1 = np.array([0.005, 0.004, 0.004])

f_s2 = np.array([0.646, 0.678, 0.728])
sig_f_s2 = np.array([0.006, 0.004, 0.006])

f_I = 1 / 2 * (f_s1 + f_s2)
f_II = 1 / 2 * (f_s2 - f_s1)
sig_f = 1 / 2 * sqrt( (sig_f_s1) ** 2 + (sig_f_s2) ** 2)

# Theoretical values
f_I_theo = 1 / 2 * (f_1 + f_2)
f_II_theo = 1 / 2 * (f_2 - f_1)
sig_f_theo = 1 / 2 * sqrt( (sig_f_1) ** 2 + (sig_f_2) ** 2 )

print('Experimentell:')
ergebnis_large('f_I', f_I, sig_f, 3, '[Hz]')
ergebnis_large('f_II', f_II, sig_f, 3, '[Hz]')

print('Theoretisch:')
ergebnis_large('f_I_theo', f_I_theo, sig_f_theo, 3, '[Hz]')
ergebnis_large('f_II_theo', f_II_theo, sig_f_theo, 3, '[Hz]')

Experimentell:
f_I = [0.634 0.65  0.674]
  +/- [0.004 0.003 0.004] [Hz]
f_II = [0.012 0.029 0.054]
   +/- [0.004 0.003 0.004] [Hz]
Theoretisch:
f_I_theo = [0.638 0.651 0.679]
       +/- [0.006 0.004 0.006] [Hz]
f_II_theo = [0.018 0.03  0.058]
        +/- [0.006 0.004 0.006] [Hz]


Coupling strengths
--

In [7]:
kappa = f_2 ** 2 / (2 * f_1 ** 2) - 1 / 2
sig_kappa = sqrt( (f_2 * sig_f_2 / (f_1 ** 2)) ** 2 
                 + (f_2 ** 2 * sig_f_1 / (f_1 ** 3)) ** 2 )

kappa_s = f_s2 ** 2 / (2 * f_s1 ** 2) - 1 / 2
sig_kappa_s = sqrt( (f_s2 * sig_f_s2 / (f_s1 ** 2)) ** 2 
                 + (f_s2 ** 2 * sig_f_s1 / (f_s1 ** 3)) ** 2 )

ergebnis_large('kappa', kappa, sig_kappa, 3, '')
ergebnis_large('kappa_s', kappa_s, sig_kappa_s, 3, '')

kappa = [0.058 0.1   0.204]
    +/- [0.019 0.013 0.024] 
kappa_s = [0.039 0.096 0.189]
      +/- [0.013 0.01  0.014] 


Coupling quotients
--

In [8]:
d = 10 + np.array([9.3, 19, 29])
sig_d = 0.2 * np.ones(3)

v_kappa = np.array([])
sig_v_kappa = np.array([])
v_d = np.array([])
sig_v_d = np.array([])
#order = np.array([])
#alist = np.array([1, 2, 3])

for i in range(len(kappa)):
    for j in range(len(kappa)):
        if i != j and i < j:
#            order = np.append(order, alist[i] / alist[j])
            v_kappa = np.append(v_kappa, kappa[i] / kappa[j])
            sig_v_kappa = np.append(sig_v_kappa, 
                                    sqrt( (sig_kappa[i] / kappa[j]) ** 2 
                                        + (kappa[i] * sig_kappa[j] 
                                           / kappa[j] ** 2) ** 2))
            

            
            
for i in range(len(d)):
    for j in range(len(d)):
        if i != j and i < j:
            v_d = np.append(v_d, d[i] ** 2 / d[j] ** 2)
            sig_v_d = np.append(sig_v_d, sqrt( (2 * d[i] * sig_d[i] / d[j] ** 2) ** 2 
                                              + (d[i] ** 2 * 2 * sig_d[j] 
                                                 / d[j] ** 3) ** 2) )
            
            
ergebnis_large('v_kappa', v_kappa, sig_v_kappa, 3, '')
ergebnis_large('v_d', v_d, sig_v_d, 3, '')

v_kappa = [0.583 0.284 0.487]
      +/- [0.208 0.1   0.086] 
v_d = [0.443 0.245 0.553]
  +/- [0.011 0.006 0.01 ] 


Sigmas
--

In [9]:
# Sanity check symmetrical oscillation
fehler('Symmetrical osc.', f_1, sig_f_1, f_free, sig_f_free)
fehler('Symmetrical osc. (beat)', f_s1, sig_f_s1, f_free, sig_f_free)

Symmetrical osc.
Relativer Fehler:  [0.80645161 0.80515298 0.80515298]
Rel. Fehler (Vergleich): 0.8064516129032258
Absoluter Fehler:  [0.    0.001 0.001]
Sigma-Abweichung:  [0.         0.14142136 0.14142136] 

Symmetrical osc. (beat)
Relativer Fehler:  [0.80385852 0.64412238 0.64516129]
Rel. Fehler (Vergleich): 0.8064516129032258
Absoluter Fehler:  [0.002 0.001 0.   ]
Sigma-Abweichung:  [0.28284271 0.15617376 0.        ] 



In [10]:
# Frequenzen f_I, II
fehler('Frequenz f_I', f_I, sig_f, f_I_theo, sig_f_theo)
fehler('Frequenz f_II', f_II, sig_f, f_II_theo, sig_f_theo)

Frequenz f_I
Relativer Fehler:  [0.61595029 0.43547762 0.53494826]
Rel. Fehler (Vergleich): [0.8768894  0.54351021 0.88976774]
Absoluter Fehler:  [-0.0035 -0.001  -0.005 ]
Sigma-Abweichung:  [0.5132649  0.22086305 0.71066905] 

Frequenz f_II
Relativer Fehler:  [32.54270698  9.9243057   6.67694681]
Rel. Fehler (Vergleich): [31.94382825 11.9848607  10.41641894]
Absoluter Fehler:  [-0.0055 -0.001  -0.004 ]
Sigma-Abweichung:  [0.80655913 0.22086305 0.56853524] 



In [11]:
# Kappas
fehler('kappa', kappa, sig_kappa, kappa_s, sig_kappa_s)

kappa
Relativer Fehler:  [33.1995471  13.13700693 11.6949143 ]
Rel. Fehler (Vergleich): [33.68889214 10.84565862  7.62052822]
Absoluter Fehler:  [0.01871539 0.00352141 0.01487652]
Sigma-Abweichung:  [0.80027512 0.21069344 0.5330823 ] 



In [12]:
# Verhältnisse
fehler('v_kappa_d', v_kappa, sig_v_kappa, v_d, sig_v_d)

v_kappa_d
Relativer Fehler:  [35.70421374 35.19916119 17.58840446]
Rel. Fehler (Vergleich): [2.48956108 2.31243526 1.71884745]
Absoluter Fehler:  [ 0.14032874  0.03929943 -0.0656536 ]
Sigma-Abweichung:  [0.67293081 0.39222824 0.76138886] 

